In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal
from prettytable import PrettyTable


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    reader = csv.reader(f)
    count = 1
    for row in reader:
        if count == 5 or 1:
            print(row)
            count += 1
            if count == 5:
                break
        else:
            count += 1
#     print(sum(1 for line in f))

['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']
['Mynt', 'Celeste', 'F', '2', 'Williams', '166.94812', 'free', 'Klamath Falls, OR', '52', 'Playa Haters', '53']
['Taylor Swift', 'Celeste', 'F', '3', 'Williams', '230.47791', 'free', 'Klamath Falls, OR', '52', 'You Belong With Me', '53']
['Amy Winehouse', 'Celeste', 'F', '4', 'Williams', '229.85098', 'free', 'Klamath Falls, OR', '52', 'Valerie', '53']


In [6]:

# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [8]:

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

    
def execute_query(query):
    try:
        return session.execute(query)
    except Exception as e:
        print(e)
        
def execute_query_with_params(query, params):
    try:
        return session.execute(query, params)
    except Exception as e:
        print(e)
        
def process_csv_file(file, func):
    """
    Process all the data executing the given func for every *.json file of the given filepath
    :param filepath: The file path
    :return:None
    """
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)
        for line in csvreader:
            func(line)

In [10]:
session.execute("""
    CREATE TABLE IF NOT EXISTS session_library (
        session_id int,
        item_in_session int,
        artist text,
        song text,
        song_length decimal,
        PRIMARY KEY (session_id, item_in_session)
    )
""")



In [11]:
def insert_to_session_library(line):
    query = "INSERT INTO session_library (session_id, item_in_session, artist, song, song_length)"
    query = query + " VALUES (%s, %s, %s, %s, %s)"
    queryParameters = (int(line[8]), int(line[3]), line[0], line[9], Decimal(line[5]))
    
    return execute_query_with_params(query, queryParameters)

process_csv_file('event_datafile_new.csv', func=insert_to_session_library)